In [1]:
%load_ext autoreload
%reload_ext autoreload

In [ ]:
def tdt_to_pandas_option2(path, t1=0, t2=0, channel=None, store='', spg = False, window_length=None, overlap=None, **kwargs):
    # Get the basic info needed from the TDT file:
    data = tdt.read_block(path, t1=t1, t2=t2, store=store, channel=channel)
    store = data.streams[store]
    info = data.info

    # Convert the TDT times to datetime objects:
    n_channels, n_samples = store.data.shape
    time = np.arange(0, n_samples) / store.fs + store.start_time
    timedelta = pd.to_timedelta(time, "s")
    datetime = pd.to_datetime(info.start_date) + timedelta
    
    # Convert this data to a pandas dataframe:
    volts_to_microvolts = 1e6
    df = pd.DataFrame(store.data.T*volts_to_microvolts, columns=["1", "2"])
    df['datetime'] = datetime
    df['timedelta'] = timedelta
    df['tdt_time'] = time
    
    # melted with multiindex
    df = df.melt(id_vars=['datetime', 'timedelta', 'tdt_time'], value_vars=["1", "2"], var_name='channel', value_name='microvolts')
    arrays = [df['datetime'], df['channel']]
    df = df.set_index(pd.MultiIndex.from_arrays(arrays))
    df.fs = store.fs
    return df


In [5]:
import tdt
import xarray as xr
import numpy as np
import pandas as pd
import kd_analysis.signal.timefrequency as tfr
import kd_analysis.main.kd_utils_pandas as kup
import kd_analysis.ACR.acr_utils as acu

In [6]:
block_path = '/Volumes/opto_loc/Data/ACR_9/ACR_9-control1'
t1 = 0
t2 = 3600
store = 'EEGr'
channel = [1,2]

In [ ]:
data = tdt.read_block(bp, t1=t1, t2=t2, store=store, channel=channel)
data.streams['EEGr'].data

In [8]:
data_df = kup.tdt_to_pandas(block_path, t1, t2, channel=[2, 8, 15], store='LFP_')
spg = kup.pd_spg(data_df)
bp = kup.pd_bp(spg)

read from t=0s to t=4759.98s
Using 610.3516 Hz as SEV sampling rate for LFP_


In [9]:
st = bp.datetime.values[0]
sp = bp.datetime.values[29]
ts = slice(st, sp)

In [12]:
st

numpy.datetime64('2022-06-14T08:50:54.999666198')

In [13]:
s1 = np.datetime64('2022-06-14T08:50:54')
s2 = np.datetime64('2022-06-14T08:51:10')
ts2 = slice(s1, s2)

In [15]:
bp.ts(ts2)

,datetime,channel,delta,theta,alpha,sigma,beta,gamma
0,2022-06-14 08:50:54.999666198,2,16182.449219,2757.188477,15670.847656,4019.144531,9039.786133,12913.102539
1,2022-06-14 08:50:54.999666198,8,17373.541016,1826.159302,12174.790039,1773.422974,2663.558105,4635.548340
2,2022-06-14 08:50:54.999666198,15,10473.057617,2861.114014,14115.025391,1741.347290,2825.706543,4897.449219
3,2022-06-14 08:50:57.000152597,2,30163.101562,7317.584961,10116.185547,8645.163086,10212.670898,17528.287109
4,2022-06-14 08:50:57.000152597,8,26321.230469,5158.116699,5834.709473,3442.412598,2525.285400,4900.593750
5,2022-06-14 08:50:57.000152597,15,30632.158203,4312.957031,7474.238281,4148.088379,3334.878906,5452.895996
6,2022-06-14 08:50:59.000638995,2,18199.750000,9552.567383,7926.820312,2823.684326,4544.212891,5889.181641
7,2022-06-14 08:50:59.000638995,8,13756.288086,6790.891602,8436.933594,1223.903076,1862.897949,4461.306641
8,2022-06-14 08:50:59.000638995,15,11402.102539,7842.079590,13165.121094,1750.487427,2740.369629,4393.437988
9,2022-06-14 08:51:01.001125393,2,19745.648438,7412.297363,6058.932617,4050.820312,5949.108887,6860.940430


In [25]:
bpi.loc[ts]

,channel,delta,theta,alpha,sigma,beta,gamma
datetime,,,,,,,
2022-06-14 08:50:54.999666198,2,16182.449219,2757.188477,15670.847656,4019.144531,9039.786133,12913.102539
2022-06-14 08:50:54.999666198,8,17373.541016,1826.159302,12174.790039,1773.422974,2663.558105,4635.548340
2022-06-14 08:50:54.999666198,15,10473.057617,2861.114014,14115.025391,1741.347290,2825.706543,4897.449219
2022-06-14 08:50:57.000152597,2,30163.101562,7317.584961,10116.185547,8645.163086,10212.670898,17528.287109
2022-06-14 08:50:57.000152597,8,26321.230469,5158.116699,5834.709473,3442.412598,2525.285400,4900.593750
2022-06-14 08:50:57.000152597,15,30632.158203,4312.957031,7474.238281,4148.088379,3334.878906,5452.895996
2022-06-14 08:50:59.000638995,2,18199.750000,9552.567383,7926.820312,2823.684326,4544.212891,5889.181641
2022-06-14 08:50:59.000638995,8,13756.288086,6790.891602,8436.933594,1223.903076,1862.897949,4461.306641
2022-06-14 08:50:59.000638995,15,11402.102539,7842.079590,13165.121094,1750.487427,2740.369629,4393.437988


# looking inside the xarray dataarray --> dataframe conversion used for the spectrogram (use if need to eventually get rid of xarray completely)

In [ ]:
tx.to_dataframe(name='spg')

In [ ]:
ordered_dims = dict(zip(tx.dims, tx.shape))
ordered_dims

In [ ]:
ds = tx._to_dataset_whole(name='spg')

In [ ]:
cols = [k for k in ds.variables if k not in ds.dims]
cols

In [ ]:
data = [
            ds._variables[k].set_dims(ordered_dims).values.reshape(-1)
            for k in cols
        ]

In [ ]:
index = ds.coords.to_index([*ordered_dims])
index

In [ ]:
spg_df = pd.DataFrame()

In [ ]:
dd = dict(zip(cols, data))
dd['timedelta'].shape

In [ ]:
pd.DataFrame(dict(zip(cols, data)), index=index)

In [ ]:
 if spg == True:
        
        time = time.min() + spg_time
        timedelta = timedelta.min() + pd.to_timedelta(spg_time, "s")
        datetime = datetime.min() + pd.to_timedelta(spg_time, "s")
        spg_df = pd.DataFrame(spg, columns=[freqs, channel], index=pd.DatetimeIndex(datetime))
        return df, spg_df

# Plotting Dev

In [1]:
import plotly.express as px

In [ ]:
px.line(bp.bp_melt().ch([1]), x='datetime', y='Bandpower', facet_row='Band')

In [ ]:
px.line(bp.reset_index(), x='datetime', y='delta', color='channel')